In [1]:
import torch
import cmw_utils

In [7]:
x_param = torch.tensor([2.0, 2.0], requires_grad=True)
y_param = torch.tensor([2.0, 2.0], requires_grad=True)

# x_loss = torch.dot(x_param, x_param) * torch.dot(y_param, y_param)
# y_loss =  - torch.dot(x_param, x_param) * torch.dot(y_param, y_param)

x_loss = torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))
y_loss = - torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))

b1 = torch.tensor([1.0, 1.0])
b2 = torch.tensor([1.0, 1.0])


result1 = cmw_utils.avp([x_loss, y_loss], [x_param, y_param], [b1, b2], transpose=False, retain_graph=True)
result2 = cmw_utils.avp([x_loss, y_loss], [x_param, y_param], [b1, b2], transpose=True, retain_graph=True)

print(result1)
print(result2)


[tensor([32.5000, 32.5000], grad_fn=<AddBackward0>), tensor([-31.5000, -31.5000], grad_fn=<AddBackward0>)]
[tensor([-31.5000, -31.5000], grad_fn=<AddBackward0>), tensor([32.5000, 32.5000], grad_fn=<AddBackward0>)]


In [12]:
x_param = torch.tensor([0.0, 2.0], requires_grad=True)
y_param = torch.tensor([0.0, 2.0], requires_grad=True)

x_loss = torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))
y_loss = - torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))

nash_list, n_iter = cmw_utils.metamatrix_conjugate_gradient(
    [x_loss, y_loss], [x_param, y_param])

new_params = cmw_utils.project_update(nash_list, [x_param, y_param])
print(new_params)

[tensor([0.0000, 0.6606]), tensor([0.0000, 0.8450])]
